### 导入相关的包

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
from torch.optim.lr_scheduler import LambdaLR,ReduceLROnPlateau
from torchvision import datasets
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.nn import TripletMarginLoss
from sklearn.neighbors import NearestNeighbors
import torch.nn.init as init
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from torchvision import transforms
from PIL import Image
import os
import random

### 部署模型，修改分类层

In [3]:
# 使用预训练参数
from models.SqueezeNet import SqueezeNet

model = SqueezeNet(version="1_1")
model.load_state_dict(torch.load('squeezenet1_1_weights.pth'))


# 冻结参数层
for param in model.parameters():
    param.requires_grad = False

# 修改最后一层

print(model.classifier[1])
final_conv = nn.Conv2d(512, 128, kernel_size=1)
init.normal_(final_conv.weight, mean=0.0, std=0.01)
init.constant_(final_conv.bias, 0.0)
model.classifier[1] = final_conv


model = model.to('cuda')



for params in model.classifier[1].parameters():
    params.requires_grad = True
    
# 解冻模型的最后1个Fire模块
for param in model.features[-1:].parameters():
    param.requires_grad = True
    
    
print(model)


Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1))
SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_act

In [4]:
test_img_001_0 = Image.open('./data_cropped/001/001_0.bmp')
test_img_001_1 = Image.open('./data_cropped/001/001_1.bmp')
test_img_001_2 = Image.open('./data_cropped/001/001_2.bmp')
test_img_003_3 = Image.open('./data_cropped/003/003_3.bmp')
test_img_007_3 = Image.open('./data_cropped/007/007_3.bmp')
test_img_003_1 = Image.open('./data_cropped/003/003_1.bmp')
test_img_003_2 = Image.open('./data_cropped/003/003_2.bmp')

lables = ["001_0", "001_1", "001_2", "003_3", "007_3", "003_1", "003_2"]

transforming = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                                 0.229, 0.224, 0.225])
        ])

aligned = []
aligned.append(transforming(test_img_001_0))
aligned.append(transforming(test_img_001_1))
aligned.append(transforming(test_img_001_2))
aligned.append(transforming(test_img_003_3))
aligned.append(transforming(test_img_007_3))
aligned.append(transforming(test_img_003_1))
aligned.append(transforming(test_img_003_2))

model.eval()
aligned = torch.stack(aligned).to('cuda')
emdeddings = model(aligned).cpu().detach()
 
print(emdeddings[0].shape)
dists = [[(e1 - e2).norm().item() for e2 in emdeddings] for e1 in emdeddings]
# pd.DataFrame(dists, columns=lables, index=lables)




torch.Size([128])


### 读取并且定制数据集

In [5]:
loss = TripletMarginLoss(margin=0.3, p=2)


# 分割三元组数据集
class TripletFaceDataset(Dataset):
    def __init__(self, image_folder,persons,transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.persons = persons

    def __getitem__(self, _):
        anchor_person = random.choice(self.persons)
        positive_person = anchor_person
        negative_person = random.choice(self.persons)
        while negative_person == anchor_person:
            negative_person = random.choice(self.persons)

        anchor_img_path = random.choice(os.listdir(
            os.path.join(self.image_folder, anchor_person)))
        positive_img_path = random.choice(os.listdir(
            os.path.join(self.image_folder, positive_person)))
        negative_img_path = random.choice(os.listdir(
            os.path.join(self.image_folder, negative_person)))

        anchor_img = Image.open(os.path.join(
            self.image_folder, anchor_person, anchor_img_path))
        positive_img = Image.open(os.path.join(
            self.image_folder, anchor_person, positive_img_path))
        negative_img = Image.open(os.path.join(
            self.image_folder, negative_person, negative_img_path))

        # 进行数据处理
        transform = self.transform

        anchor_img = transform(anchor_img)
        positive_img = transform(positive_img)
        negative_img = transform(negative_img)

        return (anchor_img, positive_img, negative_img),(anchor_person, positive_person, negative_person)

    def __len__(self):
        return len(self.persons) * 5  # 假设每个人有5张图片


# print(labels)

# 切割数据集八二分

batch_size = 32
epochs = 20
workers = 0 if os.name == 'nt' else 8


dataset = datasets.ImageFolder('./data_cropped', transform=transforming)
labels = os.listdir('./data_cropped')
print(labels)
random.shuffle(labels)
train_idx, test_idx = train_test_split(
    labels, test_size=0.2, random_state=42)


train_dataset = TripletFaceDataset(
    image_folder='./data_cropped', persons=train_idx, transform=transforming
    )

test_dataset = TripletFaceDataset(
    image_folder='./data_cropped', persons=test_idx, transform=transforming
    )

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)



['209', '085', '187', '104', '132', '436', '092', '370', '197', '497', '308', '081', '376', '427', '387', '039', '364', '128', '381', '241', '399', '314', '231', '366', '272', '217', '098', '274', '233', '024', '423', '083', '385', '222', '013', '407', '184', '299', '448', '256', '050', '315', '369', '130', '475', '212', '174', '142', '388', '492', '244', '340', '397', '208', '418', '007', '019', '305', '390', '166', '040', '037', '474', '421', '043', '065', '077', '135', '361', '260', '117', '080', '449', '118', '106', '405', '220', '298', '099', '269', '263', '120', '084', '284', '488', '071', '349', '154', '334', '482', '408', '086', '317', '447', '090', '451', '477', '277', '345', '249', '267', '450', '246', '297', '147', '079', '278', '097', '171', '355', '498', '283', '001', '012', '280', '032', '069', '392', '304', '144', '161', '044', '188', '185', '030', '243', '033', '362', '316', '219', '078', '138', '202', '133', '411', '111', '215', '074', '136', '331', '172', '207', '175'

### 开始模型训练

In [6]:
# 定制优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# def lambda_rule(epoch,max_epoch):
#     max_epoch = 20
#     return (1-epoch/max_epoch)**0.9 ##多项式衰减

# scheduler = LambdaLR(optimizer, lr_lambda=lambda_rule)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=10)


# 定制参考嵌入向量
reference_embeddings = []
reference_labels = []
model.eval()
# 遍历数据集每个文件夹，每个文件夹embeddings计算平均值，放进reference_embeddings,对应的标签放进reference_labels
for label in labels:
    label_embeddings = []
    for img_path in os.listdir(os.path.join('./data_cropped', label)):
        img = Image.open(os.path.join('./data_cropped', label, img_path))
        img = transforming(img).to('cuda')
        img = img.unsqueeze(0)
        with torch.no_grad():
            embedding = model(img).cpu().detach().numpy()
            label_embeddings.append(embedding)    
    label_embeddings = np.array(label_embeddings)
    np_mean = np.mean(label_embeddings, axis=0)
    tensor_mean = torch.from_numpy(np_mean).to('cuda')
    reference_embeddings.append(tensor_mean)
    reference_labels.append(label)



/home/lhy/anaconda3/envs/squeezenet/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


### 开始训练

In [7]:
# writer = SummaryWriter('runs/triplet_loss_experiment')
# writer.iteration, writer.interval = 0, 10

print('Start Training')
print('-'*10)


for epoch in range(epochs):
    print('\n循环 {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)
    
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        anchor, positive, negative = data[0]
        anchor, positive, negative = anchor.to('cuda'), positive.to('cuda'), negative.to('cuda')
        optimizer.zero_grad()
        anchor_out = model(anchor)
        positive_out = model(positive)
        negative_out = model(negative)
        loss_val = loss(anchor_out, positive_out, negative_out)
        loss_val.backward()
        optimizer.step()
        running_loss += loss_val.item()
        # writer.iteration += 1
        print(f"the loss is {running_loss}")
        if i % 10 == 9:
            # writer.add_scalar('loss', running_loss / 10,
            #                   writer.iteration)
            running_loss = 0.0
    print('Epoch{} finished'.format(epoch))
    
    
    model.eval()
    ## we use acc to evaluate the model
    with torch.no_grad():
        # 用标签来评估模型
        total = 0
        correct = 0
        reference_embeddings_np = np.vstack([embedding.cpu().numpy() for embedding in reference_embeddings])
        near_nn = NearestNeighbors(n_neighbors=1, algorithm='auto',metric='euclidean',).fit(reference_embeddings_np)
        # use labels to evaluate the model
        for i,data in enumerate(test_loader,0):
                anchor, _, _ = data[0]
                anchor_label, _, _ = data[1]
                batch_sizes = anchor.size(0)
                total += batch_sizes
                anchor = anchor.to('cuda')
                # print(anchor.shape)
                anchor_out = model(anchor).cpu().detach().numpy()
                
                distances, indices = near_nn.kneighbors(anchor_out)
                indices = indices.reshape(-1)
                
                for j in range(batch_sizes):
                    if reference_labels[indices[j]] == anchor_label[j]:
                        correct += 1
        acc = correct / total
        print('acc:', acc)
        print('correct:', correct)
        
        # writer.add_scalar('acc', acc, epoch)
        
        
        # 用scheduler来更新学习率
        scheduler.step(acc)
        current_lr = scheduler.get_last_lr()
        print("Current learning rate: ", current_lr)
        print('lr:', optimizer.param_groups[0]['lr'])
        
print('Finished Training')

# writer.close()




    

    
        
    
    
        
    





                
    
        
                    
                    
                
              
            
            
            


Start Training
----------

循环 1/20
----------


the loss is 0.13387802243232727
the loss is 0.2712397277355194
the loss is 0.45408427715301514
the loss is 0.6154322624206543
the loss is 0.7360809743404388
the loss is 0.8768720328807831
the loss is 1.0507438480854034
the loss is 1.1260280534625053
the loss is 1.21124629676342
the loss is 1.2634118273854256
the loss is 0.22343415021896362
the loss is 0.3251304402947426
the loss is 0.38285718113183975
the loss is 0.537703312933445
the loss is 0.5800512954592705
the loss is 0.685709722340107
the loss is 0.6984297782182693
the loss is 1.0367788225412369
the loss is 1.083562508225441
the loss is 1.2232123464345932
the loss is 0.11082091182470322
the loss is 0.28617649525403976
the loss is 0.2871822193264961
the loss is 0.3419572338461876
the loss is 0.42946167290210724
the loss is 0.5750168710947037
the loss is 0.6703439205884933
the loss is 0.7709489017724991
the loss is 0.8533849790692329
the loss is 0.9389441609382629
the loss is 0.08719602972269058
the loss is 0.16390536725521088
the 

Exception ignored in: <function _releaseLock at 0x7f1a5bd835b0>
Traceback (most recent call last):
  File "/home/lhy/anaconda3/envs/squeezenet/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


acc: 0.016
correct: 8
lr: 0.001

循环 6/20
----------
the loss is 0.023313023149967194
the loss is 0.03483925759792328
the loss is 0.0405062735080719
the loss is 0.05535769462585449
the loss is 0.1580553874373436
the loss is 0.17215154320001602
the loss is 0.17215154320001602
the loss is 0.28834231197834015
the loss is 0.28834231197834015
the loss is 0.3542257845401764
the loss is 0.09027640521526337
the loss is 0.09027640521526337
the loss is 0.09027640521526337
the loss is 0.11901484429836273
the loss is 0.2272922396659851
the loss is 0.27175746858119965
the loss is 0.27175746858119965
the loss is 0.2911672443151474
the loss is 0.38287481665611267
the loss is 0.4086630195379257
the loss is 0.0
the loss is 0.004060432314872742
the loss is 0.009870216250419617
the loss is 0.14102958142757416
the loss is 0.14102958142757416
the loss is 0.17163527011871338
the loss is 0.2632328122854233
the loss is 0.29239600896835327
the loss is 0.375115804374218
the loss is 0.4600604623556137
the loss is

KeyboardInterrupt: 